# Define functions for analysis of vacuum-ultraviolet data by the exciton and spontelectric models

In [2]:
# Import necessary modules
import os
import math
import cmath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, MultipleLocator, FormatStrFormatter
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
import scipy.constants as constants
import seaborn as sns
import sympy as sp
from sympy import symbols, diff
from collections import defaultdict
from IPython.display import Image, HTML, Markdown
import openpyxl
import xlrd
import glob
from sys import float_info

In [5]:
# Function definitions for fitting VUV spectra
#--------------------------------------------------------

#   Gaussian formula
def gaussian(x, A, x0, sig):
    return A*np.exp(-(x-x0)**2/(2*sig**2))

def multi_gaussian(x, *pars):
    
    g1 = gaussian(x, pars[0], pars[1], pars[2])
    g2 = gaussian(x, pars[3], pars[4], pars[5])
    g3 = gaussian(x, pars[6], pars[7], pars[8])
    g4 = gaussian(x, pars[9], pars[10], pars[11])    
    return g1 + g2 + g3 + g4

# Converting wavelength (nm) to energy (eV)
# Function to prevent zero values in an array
def preventDivisionByZero(some_array):
    corrected_array = some_array.copy()
    for i, entry in enumerate(some_array):
        # If element is zero, set to some small value
        if abs(entry) < float_info.epsilon:
            corrected_array[i] = float_info.epsilon
    
    return corrected_array

def WLtoE(wl):
    # Prevent division by zero error
    wl = preventDivisionByZero(wl)

    # E = h*c/wl            
    h = constants.h         # Planck constant
    c = constants.c         # Speed of light
    J_eV = constants.e      # Joule-electronvolt relationship
    
    wl_nm = wl * 10**(-9)   # convert wl from nm to m
    E_J = (h*c) / wl_nm     # energy in units of J
    E_eV = E_J / J_eV       # energy in units of eV
    
    return E_eV  

# Converting energy (eV) to wavelength (nm)
def EtoWL(E):
    # Prevent division by zero error
    E = preventDivisionByZero(E)
    
    # Calculates the wavelength in nm
    return constants.h * constants.c / (constants.e * E) * 10**9

def PlotGaussFit(name, x, data, fit, residuals_4gauss, gauss_peak_1,gauss_peak_2,gauss_peak_3,gauss_peak_4):
    
    ###
    
    fig, axs = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios':[3,1]})
    fig.subplots_adjust(hspace=0) # Remove horizontal space between axes

    fig.suptitle('fit data with 4 Gaussians'+name, family="serif", fontsize=12)
    plt.xlabel('wavelength / eV', family="serif", fontsize=12)

    axs[1].plot(x,residuals_4gauss,'go:',label='res')

    axs[1].set_ylabel("residuals",family="serif", fontsize=12)  
    axs[0].plot(x,data,'b+:',label='data')

    axs[0].plot(x, gauss_peak_1, "g")
    axs[0].fill_between(x, gauss_peak_1.min(), gauss_peak_1, facecolor="green", alpha=0.5)

    axs[0].set_ylabel("absorbance",family="serif", fontsize=12)    
    axs[0].plot(x,  gauss_peak_2, "y")
    axs[0].fill_between(x, gauss_peak_2.min(), gauss_peak_2, facecolor="yellow", alpha=0.5)  

    axs[0].plot(x,  gauss_peak_3, "m")
    axs[0].fill_between(x, gauss_peak_3.min(), gauss_peak_3, facecolor="magenta", alpha=0.5)  

    axs[0].plot(x,  gauss_peak_4, "k")
    axs[0].fill_between(x, gauss_peak_4.min(), gauss_peak_4, facecolor="k", alpha=0.5)  
    
    axs[0].plot(x,  fit, "-k", label = 'Gaussian fit')

    axs[0].annotate(residuals, xy =(150, max(df['absorbance_bk'])))
    axs[0].text(0,0.7,'pars_4'+str(pars_4),transform=axs[0].transAxes)
    axs[0].text(0,0.6,'pars_3'+str(pars_3),transform=axs[0].transAxes)
    axs[0].text(0,0.5,'pars_2'+str(pars_2),transform=axs[0].transAxes)
    axs[0].text(0,0.4,'pars_1'+str(pars_1),transform=axs[0].transAxes)

    axs[1].legend()
    axs[0].legend()

def gaussian_area(A, sig):
    return A * sig * np.sqrt(2 * np.pi)

def PlotGauss_PeakMax(dataframe):
    fig, axs = plt.subplots(4, 1, sharex=True, figsize=(4,7)) # I am making a 4 x 1 row x column grid
    fig.subplots_adjust(hspace=0.1)
    plt.xlabel('temperature / K', family="garamond", fontsize=18)
    plt.rcParams['errorbar.capsize']=2
    dataframe.loc[(dataframe.ab1>0.40) & (dataframe.ab1<0.71)].plot.scatter(x='temp',y='peak1', yerr = 'peak1_err', ax=axs[1])
    dataframe.loc[(dataframe.ab1>0.40) & (dataframe.ab1<0.71)].plot.scatter(x='temp',y='peak2', yerr = 'peak2_err',ax=axs[2])
    dataframe.loc[(dataframe.ab1>0.40) & (dataframe.ab1<0.71)].plot.scatter(x='temp',y='peak3', yerr = 'peak3_err',ax=axs[0])
    dataframe.loc[(dataframe.ab1>0.40) & (dataframe.ab1<0.71)].plot.scatter(x='temp',y='peak4', yerr = 'peak4_err',ax=axs[3])
    plt.xlim(120,145)
    plt.show()

def PlotFitSpace(dataframe):

    y_axis = np.ones(len(dataframe.peak1))
    fig, axs = plt.subplots(4, 3, sharex=False)
    fig.subplots_adjust(hspace=1)

    fig.suptitle('fit space explored', family="serif", fontsize=12)    
        
    axs[0,0].plot(p.sig1,y_axis,'ro')
    axs[0,0].set_xlim(bounds_min[2],bounds_max[2])
    axs[0,0].set_title('width')
    axs[0, 0].set_yticklabels([])
    axs[0,0].set_ylabel('144 nm')

    axs[1,0].plot(p.sig2,y_axis,'ro')
    axs[1,0].set_xlim(bounds_min[5],bounds_max[5])
    axs[1, 0].set_yticklabels([]) 
    axs[1,0].set_ylabel('155 nm')

    axs[2,0].plot(p.sig3,y_axis,'ro')
    axs[2,0].set_xlim(bounds_min[8],bounds_max[8])
    axs[2, 0].set_yticklabels([]) 
    axs[2,0].set_ylabel('BkGd')
    
    axs[3,0].plot(p.sig4,y_axis,'ro')
    axs[3,0].set_xlim(bounds_min[11],bounds_max[11])
    axs[3, 0].set_yticklabels([]) 
    axs[3,0].set_ylabel('contaminant')
    
    axs[0,1].plot(p.peak1,y_axis,'ro')
    axs[0,1].set_xlim(bounds_min[1],bounds_max[1])
    axs[0,1].set_yticklabels([]) 
    axs[0,1].set_title('pos')

    axs[1,1].plot(p.peak2,y_axis,'ro')
    axs[1,1].set_xlim(bounds_min[4],bounds_max[4])
    axs[1, 1].set_yticklabels([]) 

    axs[2,1].plot(p.peak3,y_axis,'ro')
    axs[2,1].set_xlim(bounds_min[7],bounds_max[7])
    axs[2, 1].set_yticklabels([]) 

    axs[3,1].plot(p.peak4,y_axis,'ro')
    axs[3,1].set_xlim(bounds_min[10],bounds_max[10])
    axs[3, 1].set_yticklabels([]) 
    
    axs[0,2].plot(p.ab1,y_axis,'ro')
    axs[0,2].set_xlim(bounds_min[0],bounds_max[0])
    axs[0, 2].set_yticklabels([]) 
    axs[0,2].set_title('amp')

    axs[1,2].plot(p.ab2,y_axis,'ro')
    axs[1,2].set_xlim(bounds_min[3],bounds_max[3])
    axs[1, 2].set_yticklabels([]) 

    axs[2,2].plot(p.ab3,y_axis,'ro')
    axs[2,2].set_xlim(bounds_min[6],bounds_max[6])
    axs[2, 2].set_yticklabels([]) 

    axs[3,2].plot(p.ab4,y_axis,'ro')
    axs[3,2].set_xlim(bounds_min[9],bounds_max[9])
    axs[3, 2].set_yticklabels([])

#define function to find the absorbance and energy value that corresponds to a peak position and add an offset
def find_closest_match(df, peak_values, i, offset): 
    
    # Find the row in 'df' where 'photon_energy' is closest to 'peak"x"_values[i]'
    closest_row = df.iloc[(df['photon_energy'] - peak_values[i]).abs().argsort()[:1]]
    
    # Get the 'index', 'absorbance' and 'energy' values from the closest row and add the 'offset'
    index_best_value = closest_row.index[0]
    absorbance_value_peak1 = closest_row['absorbance'].iloc[0] + offset
    energy_value_peak1 = closest_row['photon_energy'].iloc[0]
    # Return the result
    closest_xy = (index_best_value , energy_value_peak1 , absorbance_value_peak1)
    return closest_xy

# Define the exponential function to fit
def exponential_decay(x, a, b, c):
    return a * np.exp(-b * x) + c



In [20]:
# Function definitions for applying exciton model
#--------------------------------------------------------

def PlotSpontelData(xmin,xmax,Temp,sigT,ymodel_E,Eobs,annotate,xmin_mu,xmax_mu,ymodel,y,sigy,ij_space,res_E,res_mu,E_A_Vm, filename):
    fig, ax = plt.subplots(2, 2, sharex=True, gridspec_kw={'height_ratios':[3,1]}, figsize=(10, 5.4))
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.02)
        
    #plt.title(ij_space)
    ax[0,0].set_xlim(xmin, xmax)
    #ax[0,0].set_ylim(ymin, ymax)
    ax[0,0].errorbar(Temp, ymodel_E, markersize=10, xerr=sigT, yerr=None, fmt='.', color='red', label='model')
    ax[0,0].scatter(Temp, Eobs, marker='^', s=50, color='blue', label='observed')
        #ax.set_yticks(tick)
    ax[0,0].set_xlabel('Temperature / K', fontsize=16, fontname='garamond')
    ax[0,0].xaxis.set_label_position("top")
    ax[0,0].xaxis.tick_top()
    ax[0,0].set_ylabel('Electric Field / Vm$^{-1}$', fontsize=16, fontname='garamond')
    ax[0,0].tick_params(labelsize='large')
    ax[0,0].legend(loc=0, fontsize=14, framealpha=0)
    ax[0,0].text(0.25, 0.25, annotate, transform=ax[0,0].transAxes)
        
    ax[0,1].set_xlim(xmin_mu, xmax_mu)
    #ax[0,1].set_ylim(ymin_mu, ymax_mu)
    ax[0,1].errorbar(Temp, ymodel, markersize=10, xerr=sigT, yerr=sigy, fmt='.', color='red', label='model')
    ax[0,1].scatter(Temp, y, marker='^', s=50, color='blue', label='observed')
    ax[0,1].set_xlabel('Temperature / K', fontsize=16, fontname='garamond')
    ax[0,1].xaxis.set_label_position("top")
    ax[0,1].xaxis.tick_top()
    ax[0,1].yaxis.set_label_position("right")
    ax[0,1].yaxis.tick_right()
    ax[0,1].set_ylabel(r'Dipole orientation / $\mu_Z/\mu$', fontsize=16, fontname='garamond')
    ax[0,1].tick_params(labelsize='large')
    ax[0,1].legend(loc=0, fontsize=14, framealpha=0)
    ax[0,1].text(0,0.8,ij_space+'\n Calculated value of E_A is '+str(round(E_A_Vm/1E9,3))+' x 10^9 V/m',transform=ax[0,0].transAxes)
    #ax[0,1].text(20,0,legend_mu)
    
    ax[1,0].scatter(Temp, res_E, marker='+', s=40, color='red', label='observed')
    ax[1,0].axhline(y = 0.0, color = 'g', linestyle = '-')
    ax[1,0].set_xlabel('Temperature / K', fontsize=16, fontname='garamond')
    ax[1,0].set_ylabel('res', fontsize=16, fontname='garamond')
    
    ax[1,1].scatter(Temp, res_mu, marker='+', s=40, color='red', label='observed')
    ax[1,1].yaxis.set_label_position("right")
    ax[1,1].axhline(y = 0.0, color = 'g', linestyle = '-')
    ax[1,1].yaxis.tick_right()
    ax[1,1].set_xlabel('Temperature / K', fontsize=16, fontname='garamond')
    ax[1,1].set_ylabel('res', fontsize=16, fontname='garamond')
    
    plt.savefig(filename)


def PlotSpontelData_paper(Temp,sigT,ymodel_E,Eobs,ymodel,y,sigy, axa, axb):
    
    xmin = np.min(Temp) -2
    xmax = np.max(Temp) +2
    xmin_mu = xmin
    xmax_mu = xmax
        
    axa.set_xlim(xmin, xmax)
    #axa.set_ylim(ymin, ymax)
    axa.plot(Temp, Eobs, 'ko', label='exciton model')
    axa.plot(Temp, ymodel_E, marker='^', color='None', markeredgecolor='r', markerfacecolor='red', label='spontelectric model')
    axa.xaxis.set_ticks_position('both')
    axa.set_xticks(np.arange(xmin+0.5, xmax, 6))
    axa.tick_params(axis='x', which='both', length=5,  labelsize = 10, direction='inout')
    axa.xaxis.set_minor_locator(MultipleLocator(1))
    axa.tick_params(axis='x', which='minor', length=2, labelsize=0)  
    #axa.xaxis.set_label_position('bottom')
    #axa.xaxis.tick_top()
    
    axa.tick_params(axis='y', which='both', length=5,  labelsize = 10, direction='inout')
    axa.yaxis.set_tick_params(right=True, labelright=False)
    axa.yaxis.set_major_locator(MaxNLocator(nbins=4))
    #axa.legend(loc=0, fontsize=14, framealpha=0)
    #axa.text(0.25, 0.25, annotate, transform=axa.transAxes)
        
    axb.set_xlim(xmin_mu, xmax_mu)
    #axb.set_ylim(ymin_mu, ymax_mu)
    axb.plot(Temp, y, 'ko',  label='exciton model')
    axb.plot(Temp, ymodel, marker='v', color='None', markeredgecolor='r',markerfacecolor='red',label='spontelectric model')
    axb.xaxis.set_ticks_position('both')
    axb.set_xticks(np.arange(xmin+0.5, xmax, 6))
    
    #axb.set_xlabel('Temperature / K', fontsize=16, fontname='garamond')
    axb.tick_params(axis='x', which='both', length=5,  labelsize = 10, direction='inout')
    axb.xaxis.set_minor_locator(MultipleLocator(1))
    axb.tick_params(axis='x', which='minor', length=2, labelsize=0)  
    axb.yaxis.set_label_position("right")
    axb.tick_params(axis='y', which='both', length=5,  labelsize = 10, direction='inout')
    axb.yaxis.set_tick_params(right=True, labelleft = False, labelright=True)
    #axb.legend(loc=0, fontsize=14, framealpha=0)   
    #plt.savefig(filename)

###
### - Define some functions for the exciton model
###

def Solid_phase_dipole(density):
    
    #mu_0_Debye = 3 #fixed for water
    mu_0 = mu_0_Debye/2.54158025
            
    return mu_0
        
def Observed_Esp_David(radius, DE): # r in au
#
# gives the observed value of Espont for some value of radius and the observed value of DE
# NOTE that the value of DE is defined by the measured peak position and the estimated value of lambda_0 
    
    Esp_David = []
    for i in DE:
        Esp_au_David_individual = (1/96)*(-((32*i)/radius)+((16*(2**(1/3))*i*(6+i*eps*radius))/(radius**3 * ((((i**2*eps**2 *radius**(0.5))*(-9+2*i*eps*radius))+3*(3**(0.5))*(-i**3 *eps**3 *(32+i*eps*radius*(13+4*i*eps*radius)))**0.5)/(radius**(9/2)))**(1/3)))+((8*(2**(2/3))*((((i**2 *eps**2 *radius**(0.5))*(-9+2*i*eps*radius))+3*(3**(0.5))*(-i**3 *eps**3 *(32+i*eps*radius*(13+4*i*eps*radius)))**0.5)/(radius**(9/2)))**(1/3))/(eps)))
        #if DE is positive then Esp_au is a complex number and we only want the real solution:
        Esp_David = np.append(Esp_David, 5.14220652e+11*Esp_au_David_individual.real)#Esp_au_David*5.14220652e+11)
        
    return Esp_David

def function_p(r, DE): # r in au
    pee = (r*DE)
    
    return pee

def function_f(r): # r in au
    function_f = 1/12/r/r
    
    return function_f

def function_g(r,DE): # r in au
    r = float(r)
    function_g=[]
    for i in DE:       
        var_D = (((eps**2)*(function_p(r,i)**2)*(-9.0+2.0*(eps*function_p(r,i))))+3.0*((3.0)**(0.5))*(-(eps**3)*(function_p(r,i)**3)*(32.0+(eps*function_p(r,i))*(13.0+4.0*(eps*function_p(r,i)))))**(1/2))**(1/3)
        function_g_individual = -4.0*function_p(r,i)+((2.0*(2.0**(1/3))*function_p(r,i)*(6.0+function_p(r,i)*eps))/var_D)+(((2.0**(2/3))*var_D)/eps)
        function_g=np.append(function_g,function_g_individual.real)
           
    return function_g
    
def Observed_Esp(r,DE): # r in au
#
# gives the observed value of Espont for some value of radius and the observed value of DE, using the method developed by Frank
# NOTE that the value of DE is defined by the measured peak position and the estimated value of lambda_0 
 
    Esp_au= function_f(r)*function_g(r,DE) #in au
    Esp = (Esp_au*5.14220652e+11) #convert observed E_sp to V/m
    
    return Esp

###
###  - Define the relevant functions for the Spont model
###

def Langevin(x):
#
# the Langevin function for a single real argument x
# for small absolute values of x (<1.0e-6) an approximation is used that is
# correct to second order, i.e. the error term is O(eps^3)
#
    #print('argument sent to Langevin ',x)
    if (abs(x)>1.0e-6) :
        #print('running coth function')
        y=math.cosh(x)/math.sinh(x)-1/x
    else :
        y=x/3.0
    return y

def DerLangevin(x):
#
# the derivative of the Langevin function for a single real argument x
# for small absolute values of x (<1.0e-6) an approximation is used that is
# correct to second order, i.e. the error term is O(eps^2)
#
    if (abs(x)>1.0e-6) :
        auxden=1.0/math.sinh(x)
        y=-1.0*auxden*auxden+1.0/x/x
    else :
        y=1.0/3.0
    return y

def DDerLangevin(x):
#
# the second derivative of the Langevin function for a single real argument x
# for small absolute values of x (<1.0e-6) an approximation is used that is
# correct to second order, i.e. the error term is O(eps^2)
#
    if (abs(x)>1.0e-6) :
        auxden=1.0/math.sinh(x)
        y=2.0*(math.cosh(x)*auxden*auxden*auxden-1.0/x/x/x)
    else :
        y=0.0
    return y

def LangevInv(y):
#
# the inverse of the Langevin function for a single real argument y
# this uses an expression derived by Petrosyan (2017)
# 02/06/2022 (FPP): the approximation is only correct for positive values of the argument, so
#   in order to also work for negative values I have modified the function.
#
    if (abs(y)<1.0) :
        yab=abs(y)
        x=y*(3.0+yab*math.sin(7.0*yab/2.0)/5.0+yab*yab/(1-yab))
        return x
    else :
        print("LangevInv: argument out of range, stopping.")
        return

def DerLangevInv(y):
#
# the derivative of the inverse of the Langevin function for a single real argument y
# this uses an expression for the inverse of the Langevin function derived by Petrosyan (2017)
# 02/06/2022 (FPP): the approximation is only correct for positive values of the argument, so
#   in order to also work for negative values I have modified the function.
#
    if (abs(y)<1.0) :
        yab=abs(y)
        x=3.0+2.0*yab*math.sin(7.0*yab/2.0)/5.0+7.0*yab*yab*math.cos(7.0*yab/2.0)/10.0
        x+=(3.0-2.0*yab)*yab*yab/(1-yab)/(1-yab)
        return x
    else :
        print("DerLangevInv: argument out of range, stopping.")
        return
    
def NeaterOut(x,s):
#
# prepares the strings for providing nicer-looking output of parameters with uncertainties
    
    spon=math.floor(math.log10(abs(s)))+1
    divis=math.exp(spon*math.log(10))
    xout=x/divis
    sout=s/divis
    string=r'({0:.10f}$\pm${1:.2f}) $10^{2:n}$'.format(xout,sout,spon)
    return string

def ConstrLSQ(x,y,wht):
#
# Solve a linear least squares problem for 1 dependent variable y as a function of
# nvar independent variables x.
# Linear least-squares problems for more than 1 parameter require a matrix inversion
# Here for arbitrary numbers of parameters the required matrix elements are 
# calculated and the matrix is inverted, after which the parameters are obtained.
# Input: a matrix x[nvar,ndat] containing ndat measurements of nvar independent
#           variables x
#        a vector y[ndat] containing the dependent variable
#        a vector of weights [ndat] which is used to give particular measurements 
#           less weight if desired. 
# NOTE: if a constant offset is required as one of the free parameters, an x-variable 
# which is all 1's must be present in (added to) the input matrix x.
#
    if (x.shape[1] != y.shape[0]) :
        print("ConstrLSQ: unequal array lengths, stopping.")
        par=np.array([])
        return
    if (x.shape[1] != wht.shape[0]) :
        print("ConstrLSQ: unequal array lengths, stopping.")
        par=np.array([])
        return

#
# declaration of local variables
    ndat=x.shape[1]
    nvar=x.shape[0]
    whtsum=0.0
    ysum=0.0
    xsum=np.array([0.0]*nvar)
    xysum=np.array([0.0]*nvar)
    xsqsum=np.array([[0.0]*2,[0.0]*2])
    Ainv=np.array([[0.0]*2,[0.0]*2])
    par=np.array([0.0]*nvar)
#
# weighted sum of measurements of dependent variable 
    for i in range(0,ndat):
        ysum+=y[i]*wht[i]
#
# weighted sum of measurements of independent variables and of cross-products on independent variables
# with each other and with dependent variable.
    for k in range(0,nvar):
        for i in range(0,ndat):
            whtsum+=wht[i]
            xsum[k]+=x[k,i]*wht[i]
            xysum[k]+=x[k,i]*y[i]*wht[i]
        for l in range(0,nvar):
            for i in range(0,ndat):
                xsqsum[k,l]+=x[k,i]*x[l,i]*wht[i]
#
# start solving the set of linear equations for the linear least squares problem
    if (nvar==1) :
        #
# if only one parameter to be determined
        par[0]=ysum/xsum[0]
    else:
#
# for determination of 2 parameters
        if (nvar==2) :
            Det=xsqsum[0,0]*xsqsum[1,1]-xsqsum[1,0]*xsqsum[0,1]
            if (abs(Det)>1.0e-9) :
                Ainv[0,0]=xsqsum[1,1]/Det
                Ainv[0,1]=-1.0*xsqsum[0,1]/Det
                Ainv[1,0]=-1.0*xsqsum[1,0]/Det
                Ainv[1,1]=xsqsum[0,0]/Det
                par[0]=Ainv[0,0]*xysum[0]+Ainv[0,1]*xysum[1]
                par[1]=Ainv[1,0]*xysum[0]+Ainv[1,1]*xysum[1]
                
                #print('Es is ',par[0]*5.142e11,'\nzeta is ',par[1]/par[0])
            
            else :
                print("ConstrLSQ: singular matrix. stopping")
                par=np.array([])
                return
        else :
#
# the general case of 3 or more parameters to be determined.
            Ainv=np.linalg.inv(xsqsum)
            par=np.dot(Ainv,xysum)
            #
            
    return par

# Define several conversion factors to/from atomic units
# conversion of Temp to a.u. : T [a.u.]= T [K]/factor
Ttoau=315770.0
# conversion of D to a.u. : 1 D (=Debye) = factor [a.u.]
Debye=0.393456
# Convert el. field in a.u. To V/m : E [a.u.] = E [V/m]/factor
Etoau=5.142e11
#


In [6]:
#To apply the spontelectric model to some observed data

def calc_Esp(r_au, DE): # r in au

    
    ### First; run the exciton model to find observed values    
    
    # Initialize 2D arrays to store the results
    Observed_field_David = np.zeros(len(DE))
    Observed_field = np.zeros(len(DE))
    Observed_alignment = np.zeros(len(DE))
    SpontelData = pd.DataFrame(columns=['weight', 'y', 'sigma_y', 'calc_y','Temperature', 'sigma_T', 'Observed_field','Calc_field'])
    SpontelData['Temperature'] = xdataraw
    SpontelData['sigma_T'] = [1] * len(SpontelData['Temperature'])
    
    ## Convert Esp to muz/mu0 values using a fixed value for EA 

    ## Define E_A
    # Find the value of mu for this ice 
    IceMu = Solid_phase_dipole(density)*2.541746
    
    E_A=-4*np.pi*(Solid_phase_dipole(density))/Omega
    E_A_Vm = E_A*5.14220652e+11
    #print('Calculated value of E_A for '+name+' is ', E_A_Vm/1E9, 'x 10^9 V/m')

    # Get observed values for a starting value of r in au
    Observed_field = Observed_Esp(r_au, DE)       
    Observed_alignment = Observed_field/E_A_Vm       
        
    #store these outputs in dataframes
    SpontelData['Observed_field'] = (Observed_field)
    SpontelData['y'] = (Observed_alignment)
    SpontelData['sigma_y'] = abs(SpontelData['y'] * 0.01)
    #print(SpontelData)
    
    SpontelData['weight'] = [1] * len(SpontelData['Temperature'])  

    ###
    ### - then run spontelectric model to find calculated values
    ###
        
    #Only need htfc if zeta is not constant with temperature
    htfc=np.array([0.0]*4)    
    htfc[0]=1.0
    htfc[1]=0.0
    htfc[2]=0.0
    htfc[3]=0.0
    
    # Provide a first guess value for the parameter E_A
    EAnxt= E_A_Vm #take the calculated value of EA in V/m

    weight=np.array(SpontelData.weight)
    ndata=len(weight)

    # here comes a block of variable declarations
    whtsum=0.0
    yinvmn=0.0
    hTfunc=np.array([0.0]*ndata)
    DerhTfunc=np.array([0.0]*ndata)
    DerGT=np.array([0.0]*ndata)
    DerGy=np.array([0.0]*ndata)
    PartDerv=np.array([[0.0]*ndata,[0.0]*ndata,[0.0]*ndata,[0.0]*ndata])
    xdat=np.array([[0.0]*ndata,[0.0]*ndata])
    ydat=np.array([0.0]*ndata)
    zdat=np.array([0.0]*ndata)
    ymodel=np.array([0.0]*ndata)
    DParDy=np.array([[0.0]*ndata,[0.0]*ndata,[0.0]*ndata])
    DParDT=np.array([[0.0]*ndata,[0.0]*ndata,[0.0]*ndata])
    SigPar=np.array([0.0]*3)   
    
    # y is the same as <mu_Z>/mu, Temp is the deposition temperature in [K]
    # sig are the 1-sigma measurement errors
    # E observed is the observed Electric field in units [V/m]
    y=np.array(SpontelData.y)
    sigy=np.array(SpontelData.sigma_y)
    Temp=np.array(SpontelData.Temperature)
    sigT=np.array(SpontelData.sigma_T)
    Eobs=np.array(SpontelData.Observed_field)

    # Auxiliary functions f_0, f_1, f_2, and f_3 are needed as input for linear least-squares
    # These are named here as PartDerv[0,], ..., PartDerv[3,] because these are also the
    # partial derivatives of the function G with respect to T and y.
    for i in range(0,ndata):
        yinvmn+=weight[i]/y[i]
        whtsum+=weight[i]
        hTfunc[i]=htfc[0]+Temp[i]*(htfc[1]+Temp[i]*(htfc[2]+Temp[i]*htfc[3]))
        DerhTfunc[i]=htfc[1]+Temp[i]*(2.0*htfc[2]+3.0*Temp[i]*htfc[3])
        PartDerv[0,i]=(1-y[i])*LangevInv(y[i])
        PartDerv[1,i]=Ttoau*Debye*IceMu*(y[i]-1.0)/Temp[i]
        PartDerv[2,i]=-1.0*PartDerv[1,i]*y[i]
        PartDerv[3,i]=PartDerv[1,i]*y[i]*y[i]*hTfunc[i]
    #
    # start the iteration to get the parameters, initialise a choice for E_A
    yinvmn=yinvmn/whtsum
    
    EAcur=1.1*EAnxt
    itcnt=0
    while ((abs(EAnxt/EAcur-1.0)>1.0e-4) and (itcnt<20)) :
        itcnt+=1
        EAcur=EAnxt
        #print('EAcur is ',EAcur)
        for i in range(0,ndata):
            xdat[0,i]=PartDerv[1,i]
            xdat[1,i]=PartDerv[3,i]
            
            ydat[i]=-1.0*(PartDerv[0,i]+EAcur*PartDerv[2,i]/Etoau)
        #print('xdat',xdat,
        #      '\nydat ',ydat)    
        Constrout=ConstrLSQ(xdat,ydat,weight)
        #print('Constrout',Constrout*Etoau)
    
    # E_S and zeta always are forced to be >= 0
        EScur=abs(Constrout[0]*Etoau)
        ESzetacur=abs(Constrout[1]*Etoau)
                
        ratiolog=0.0
        for i in range(0,ndata) :
            Ezmod=(EScur-EAcur*y[i]+ESzetacur*hTfunc[i]*y[i]*y[i])*IceMu*Ttoau*Debye/Etoau/Temp[i]
            ratiolog+=weight[i]*(math.log(abs(EAcur*Langevin(Ezmod)))-math.log(Eobs[i]))
            DerGy[i]=(EScur-ESzetacur*hTfunc[i]*y[i]*(2.0-3.0*y[i])+EAcur*(1.0-2.0*y[i]))*IceMu*Ttoau*Debye/Etoau/Temp[i]
            DerGy[i]+=-1.0*LangevInv(y[i])+(1.0-y[i])*DerLangevInv(y[i])
            DerGT[i]=(EScur*(1-y[i])-EAcur*y[i]*(1-y[i])+ESzetacur*hTfunc[i]*y[i]*y[i]*(1-y[i]))*IceMu*Ttoau*Ttoau*Debye/Etoau/Temp[i]/Temp[i]
            DerGT[i]+=-1.*ESzetacur*DerhTfunc[i]*y[i]*y[i]*(1-y[i])*IceMu*Ttoau*Debye/Etoau/Temp[i]
        ratiolog=ratiolog/whtsum
                
        if ((Constrout[0]<0) and (EAcur<0) and (yinvmn<0) and (ratiolog>0)) :
            
            EAnxt=EAcur+2.0*yinvmn*EScur
            
        else :
            #exp_ratiolog = (math.exp(ratiolog))  ##invetn
            #print(exp_ratiolog) ##invetn
            #if 0.1 <= exp_ratiolog <= 500:##invetn
            EAnxt=EAcur/math.exp(ratiolog) #on this condition, iterate
            #else:##invetn
             #   EAnxt = EAcur*1.1 ##invetn
            
        if (itcnt>20) : 
            print ("iteration count exceeded: not converged")
            
    # also calculate the quality of the fit (chi-square) dlsqsum

    dlsqsum=0
        
    for i in range(0,ndata) :
        #print('converged result before sending to langevin \ny[i] ',y[i], '\nIceMu ',IceMu,'\nEScur ',str(EScur),'\nEAcur ',str(EAcur),'\nESzetacur ',str(ESzetacur),'\nHTfunc[i] ',hTfunc[i],'\nTemp ',Temp[i])
        ymodel[i]=Langevin((EScur-EAcur*y[i]+ESzetacur*hTfunc[i]*y[i]*y[i])*IceMu*Ttoau*Debye/Etoau/Temp[i])
        DelMod=y[i]-ymodel[i]
        dlsqsum+=weight[i]*DelMod*DelMod/sigy[i]/sigy[i]
            #print(i,weight[i]*DelMod*DelMod/sigy[i]/sigy[i])
        DParDy[0,i]=-1.0*DerGy[i]/PartDerv[1,i]
        DParDy[1,i]=-1.0*DerGy[i]/PartDerv[2,i]
        DParDy[2,i]=-1.0*DerGy[i]/PartDerv[3,i]
        DParDT[0,i]=-1.0*DerGT[i]/PartDerv[1,i]
        DParDT[1,i]=-1.0*DerGT[i]/PartDerv[2,i]
        DParDT[2,i]=-1.0*DerGT[i]/PartDerv[3,i]
    #
    dlsqsum=dlsqsum/(whtsum-2)
    # calculate the 1-sigma uncertainties for the parameters
    SigPar=np.array([0.0]*3)
    for k in range(0,3):
        for i in range(0,ndata) :
            SigPar[k]+=weight[i]*(DParDy[k,i]*DParDy[k,i]*sigy[i]*sigy[i]+DParDT[k,i]*DParDT[k,i]*sigT[i]*sigT[i]/Ttoau/Ttoau)
        SigPar[k]=Etoau*math.sqrt(SigPar[k]/whtsum)
  
    zetacur=ESzetacur/EScur
    ymodel_E=EAcur*ymodel
   
    SpontelData['calc_y'] = ymodel
    SpontelData['Calc_field'] = ymodel_E

    #Define text to print on each graph
    SigPar[2]=zetacur*math.sqrt(SigPar[2]*SigPar[2]/ESzetacur/ESzetacur+SigPar[0]*SigPar[0]/EScur/EScur)
    string = ""
    string = 'Spont fit parameters - '
    #string+=str(dataset)
    string+="\n"+"$E_S =$"+NeaterOut(EScur,SigPar[0])
    string+="\n"+"$E_A =$"+NeaterOut(EAcur,SigPar[1])
    string+="\n"+r"$\zeta =$"+NeaterOut(zetacur,SigPar[2])
    string+="\n\n"+r"reduced $\chi^2 =$"+'{0:.1f}'.format(dlsqsum)+"\n"
    annotate=(string)
    
    

    #save the fitting parameters to csv file
    parameter_frame = pd.DataFrame({'E_a': [EAcur],'E_s': [EScur],'zeta': [zetacur]})
    folder_path = "observed_data"
    os.makedirs(folder_path, exist_ok=True)
    r_i=(r_au/18.897259886)
    parameter_file_name = os.path.join(folder_path, which_peak+'_'+str(r_i) + '_parameters.csv')
    parameter_frame.to_csv(parameter_file_name, index=False)  
    print("radius used: ",r_i, "nm")
    print(annotate)
        
    #define residuals for plot (just for visualising)
    res_E=Eobs-ymodel_E
    res_mu=y-ymodel
    
    ES=NeaterOut(EScur,SigPar[0])
    EA=NeaterOut(EAcur,SigPar[1])
    zeta=NeaterOut(zetacur,SigPar[2])
    
    SpontelData['DE_eV'] = DE_eV
        
    #save the data frame
    spont_file_name = os.path.join(folder_path, which_peak+'_'+str(r_i) + '_spont.csv')
    # Use the concatenated string as the file path
    SpontelData.to_csv(spont_file_name, index=False)
        
    ### Visualise the outputs
    ###-------------------
    #Can print the table of fits here:
    #print(SpontelData)
    # And or plot here:
    #PlotSpontelData(xmin,xmax,Temp,sigT,ymodel_E,Eobs,annotate,xmin_mu,xmax_mu,ymodel,y,sigy,ij_space,res_E,res_mu,E_A_Vm, filename)   
    #PlotSpontelData_paper(Temp,sigT,ymodel_E,Eobs,ymodel,y,sigy,'spont_paper_figure.png')
    
    #print('finished spont fit model')  
    return(SpontelData['Calc_field'])
    